<a href="https://colab.research.google.com/github/jako323/Most-Streamed-Songs-with-Python-Machine-Learning/blob/regression_for_bpm/KEY_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
## parameters  key and bpm have a least 20% infulence over the songs streams
# predict key and bpm regargding all the other parameters
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import date

from google.colab import drive

drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/MAIN_DATASET_spotify2023.csv")

data.head()
encoder = LabelEncoder()
data["key"] = encoder.fit_transform(data["key"])
data["mode"] = encoder.fit_transform(data["mode"])
released_day_of_year = []

for i in range(len(data.index)):
    day_of_year = date(data['released_year'][i],
                    data['released_month'][i],
                    data['released_day'][i]).timetuple().tm_yday
    released_day_of_year.append(day_of_year)

# add new series to dataframe
data.insert(len(data.columns), "released_day_of_year", released_day_of_year, True)

data = data[["released_day_of_year", 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%','acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
X = data.drop("key", axis = 1)
y = data["key"]

X_train, X_help, y_train, y_help = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_help, y_help, test_size=0.5, random_state=42)

val_percent = (len(X_val)/(len(data)))*100
test_percent = (len(X_test)/(len(data)))*100
train_percent = (len(X_train)/(len(data)))*100
print(f"Percent of Train: {train_percent:.2f} ,Validation:{val_percent:.2f} and Test dataset:{test_percent:.2f} ")


Percent of Train: 59.98 ,Validation:19.96 and Test dataset:20.06 


LINEAR REGRESSOR

In [12]:
reg = LinearRegression()
weights_train = np.ones_like(y_train)
np.random.seed(42)
random_weights = np.random.rand(len(y_train))

reg.fit(X_train, y_train, sample_weight=weights_train)
y_pred_train = reg.predict(X_train)
y_pred_val = reg.predict(X_val)

rmse_train = np.sqrt(np.mean((y_pred_train - y_train)**2))
mean_train = mean_squared_error(y_train, y_pred_train)

rmse_val = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean_val = mean_squared_error(y_val, y_pred_val)

print(f"Rmse_val: {rmse_val}, Mean_val:  {mean_val},\nRmse_train: {rmse_train}, Mean_train:  {mean_train}")

Rmse_val: 3.389160239975793, Mean_val:  11.486407132232774,
Rmse_train: 3.5408193662010623, Mean_train:  12.537401784064492


NEURAL REGRESSOR

In [41]:
reg = MLPRegressor(hidden_layer_sizes=(5, 6, 5), random_state=50, max_iter=50, solver='sgd')
reg.fit(X_train, y_train)
y_pred_train = reg.predict(X_train)
y_pred_val = reg.predict(X_val)

# and then try some customization, for example:
#reg = MLPRegressor(hidden_layer_sizes=(15, 10, 5), random_state=1, max_iter=1000, solver='adam')

#hidden_layer_sizes=(5, 100, 500) 75% "sgd" 81%
#max_iter 11 best 83%

rmse_train = np.sqrt(np.mean((y_pred_train - y_train)**2))
mean_train = mean_squared_error(y_train, y_pred_train)
rmse_val = np.sqrt(np.mean((y_pred - y_val)**2))
mean_val = mean_squared_error(y_val, y_pred)

print(f"Rmse_val: {rmse_val}, Mean_val:  {mean_val},\nRmse_train: {rmse_train}, Mean_train:  {mean_train}")

Rmse_val: 3.5903833977448785, Mean_val:  12.89085294280206,
Rmse_train: 3.608979687070966, Mean_train:  13.024734381690847


POLYNOMIAL REGRESSOR

In [17]:
# Create polynomial features
poly_features = PolynomialFeatures(degree = 2)
X_train_poly = poly_features.fit_transform(X_train)
X_val_poly = poly_features.fit_transform(X_val)

reg = LinearRegression()
reg.fit(X_train_poly, y_train)

y_pred_train = reg.predict(X_train_poly)
y_pred = reg.predict(X_val_poly)

rmse_val = np.sqrt(np.mean((y_pred - y_val)**2))
mean_val = mean_squared_error(y_val, y_pred)

rmse_train = np.sqrt(np.mean((y_pred_train - y_train)**2))
mean_train = mean_squared_error(y_train, y_pred_train)

print(f"Rmse_val: {rmse_val}, Mean_val:  {mean_val}")
print(f"Rmse_train: {rmse_train}, Mean_train:  {mean_train}")

Rmse_val: 3.5903833977448785, Mean_val:  12.89085294280206
Rmse_train: 3.4203317267740907, Mean_train:  11.698669121177433


SVM

In [49]:
from sklearn import svm

reg = svm.SVC(kernel='linear', C = 2, max_iter = 50000 )
weights_train = np.ones_like(y_train)
np.random.seed(42)
random_weights = np.random.rand(len(y_train))

reg.fit(X_train, y_train, sample_weight=weights_train)
y_pred_train = reg.predict(X_train)
y_pred_val = reg.predict(X_val)

rmse_train = np.sqrt(np.mean((y_pred_train - y_train)**2))
mean_train = mean_squared_error(y_train, y_pred_train)

rmse_val = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean_val = mean_squared_error(y_val, y_pred_val)

print(f"Rmse_val: {rmse_val}, Mean_val:  {mean_val},\nRmse_train: {rmse_train}, Mean_train:  {mean_train}")

Rmse_val: 5.146792562772521, Mean_val:  26.489473684210527,
Rmse_train: 4.990709407017099, Mean_train:  24.907180385288967


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
